In [4]:
import numpy as np
import csv
import igl
import math
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geopandas as gpd
from shapely.ops import triangulate
import os
import pymesh
import meshio

In [24]:
import geopandas as gpd
from shapely.ops import triangulate
import os
import pymesh
import meshio

# Load the shapefile using GeoPandas
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "ocean")
path = os.path.join(folder_path, "ne_10m_ocean.shp")
shapefile = gpd.read_file(path)

# Iterate over each MultiPolygon in the shapefile
for multi_polygon in shapefile.geometry:

    # Iterate over each Polygon in the MultiPolygon
    for polygon in multi_polygon.geoms:

        # Triangulate the polygon boundary using the Delaunay algorithm
        triangles = triangulate(polygon.boundary)

        # Create a Meshio mesh object from the triangulation
        mesh_data = {"vertices": polygon.boundary, "cells": [("triangle", triangles)]}
        mesh = meshio.Mesh(mesh_data)

        # Convert the Meshio mesh object to a PyMesh mesh object
        pymesh_mesh = pymesh.form_mesh(mesh.points, mesh.cells["triangle"])

        # Generate a refined mesh on the sphere using PyMesh
        refined_mesh, _ = pymesh.split_long_edges(pymesh_mesh, 1e-5)
        refined_mesh = pymesh.resolve_self_intersection(refined_mesh)
        refined_mesh = pymesh.compute_outer_hull(refined_mesh)
        refined_mesh, _ = pymesh.collapse_short_edges(refined_mesh, 1e-5)

        # Save the mesh to a file
        output_path = "path/to/output.obj"
        pymesh.save_mesh(output_path, refined_mesh)


TypeError: Mesh.__init__() missing 1 required positional argument: 'cells'